In [ ]:
!pip install tokenizers transformers

Код взят из [данного](https://github.com/dpfried/incoder) репозитория

In [ ]:
from typing import List

import torch
import tokenizers
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
tokenizers_version = tuple(int(n) for n in tokenizers.__version__.split('.'))
if tokenizers_version < (0, 12, 1):
    print("warning: Your tokenizers version looks old and you will likely have formatting issues. We recommend installing tokenizers >= 0.12.1")

BIG_MODEL = False

CUDA = True

VERBOSE = False

if BIG_MODEL:
    model_name = "facebook/incoder-6B"
    if CUDA:
        kwargs = dict(
            revision="float16", 
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
        )
    else:
        kwargs = dict(
            low_cpu_mem_usage=True,
        )
else:
    model_name = "facebook/incoder-1B"
    kwargs = {}

print("loading model")
model = AutoModelForCausalLM.from_pretrained(model_name, **kwargs)
print("loading tokenizer")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("loading complete")

if CUDA:
    model = model.half().cuda()

BOS = "<|endoftext|>"
EOM = "<|endofmask|>"

def make_sentinel(i):
    return f"<|mask:{i}|>"

def generate(input: str, max_to_generate: int=128, temperature: float=0.2):
    input_ids = tokenizer(input, return_tensors="pt").input_ids
    if CUDA:
        input_ids = input_ids.cuda()
    max_length = max_to_generate + input_ids.flatten().size(0)
    if max_length > 2048:
        print("warning: max_length {} is greater than the context window {}".format(max_length, 2048))
    with torch.no_grad():
        output = model.generate(input_ids=input_ids, do_sample=True, top_p=0.95, temperature=temperature, max_length=max_length)
    detok_hypo_str = tokenizer.decode(output.flatten(), clean_up_tokenization_spaces=False)
    if detok_hypo_str.startswith(BOS):
        detok_hypo_str = detok_hypo_str[len(BOS):]
    return detok_hypo_str

def infill(parts: List[str], max_to_generate: int=128, temperature: float=0.2, extra_sentinel: bool=True, max_retries: int=1):
    assert isinstance(parts, list)
    retries_attempted = 0
    done = False

    while (not done) and (retries_attempted < max_retries):
        retries_attempted += 1
        if VERBOSE:
            print(f"retry {retries_attempted}")
        if len(parts) == 1:
            prompt = parts[0]
        else:
            prompt = ""
            for sentinel_ix, part in enumerate(parts):
                prompt += part
                if extra_sentinel or (sentinel_ix < len(parts) - 1):
                    prompt += make_sentinel(sentinel_ix)
        infills = []
        complete = []
        done = True
        for sentinel_ix, part in enumerate(parts[:-1]):
            complete.append(part)
            prompt += make_sentinel(sentinel_ix)
            completion = generate(prompt, max_to_generate, temperature)
            completion = completion[len(prompt):]
            if EOM not in completion:
                if VERBOSE:
                    print(f"warning: {EOM} not found")
                completion += EOM
                done = False
            completion = completion[:completion.index(EOM) + len(EOM)]
            infilled = completion[:-len(EOM)]
            infills.append(infilled)
            complete.append(infilled)
            prompt += completion
        complete.append(parts[-1])
        text = ''.join(complete)

    if VERBOSE:
        print("generated text:")
        print(prompt)
        print()
        print("parts:")
        print(parts)
        print()
        print("infills:")
        print(infills)
        print()
        print("restitched text:")
        print(text)
        print()
    
    return {
        'text': text, # str, the completed document (with infills inserted)
        'parts': parts, # List[str], length N. Same as passed to the method
        'infills': infills, # List[str], length N-1. The list of infills generated
        'retries_attempted': retries_attempted, # number of retries used (if max_retries > 1)
    } 

def docstring_to_code(example, max_to_generate=128, temperature=0.2):
    parts = example.split("<insert>")
    result = infill(parts, max_to_generate=max_to_generate, temperature=temperature)
    print("completed document:")
    return result["text"]

# Примеры:

In [10]:
example = '''\
def <insert>
    """ number to power """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def power(n, p):
    """ number to power """
    
    return n ** p
<|/ file |>


In [12]:
example = '''\
def <insert>
    """ decimal logarithm of a number """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def log_decimal(number):
    """ decimal logarithm of a number """
    number = float(number)
    return math.log10(number)
<|/ file |>


В этом примере стоит отметить, что модель использовала встроенную библиотеку питона, но, к сожалению не импортировала её

In [17]:
example = '''\
def <insert>
    """ decimal logarithm of a number, use modules and import him """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def log_decimal(number):
    """ decimal logarithm of a number, use modules and import him """
    number = float(number)
    return math.log10(number)
<|/ file |>


Просьба импортировать модули никак не повлияла на результат, хотя может быть дело в том, что это встроенная библиотека, попробуем использовать другую

In [20]:
example = '''\
def <insert>
    """ decimal logarithm of a number with module numpy """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def log_decimal(number):
    """ decimal logarithm of a number with module numpy """
    import numpy as np
    return np.log(number) / np.log(10)
<|/ file |>


Хоть решение и не вышло оптимальным, стоит отметить, что модель знает о функционале устанавливаемых модулей и импортирует их.

Попробуем что-нибудь потяжелее

In [21]:
example = '''\
def <insert>
    """ binary search algorithm """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def binary_search(arr, val):
    """ binary search algorithm """
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == val:
            return mid
        elif arr[mid] < val:
            right = mid
        else:
            left = mid
    return -1
<|/ file |>


К сожалению в алгоритме ошибка, из-за чего поиск идёт в противоположнуюю сторону от искомого элемента, попробуем дополнить описание тестами

In [22]:
example = '''\
def <insert>
    """ binary search algorithm """
    <insert>

assert binary_search([1,2,3,4,5,6], 2) == 1
assert binary_search([1,2,3,4,5,6], 5) == 4
assert binary_search([1,2,3,4,5,6], 7) == -1
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def binary_search(arr, val):
    """ binary search algorithm """
    low = 0
    high = len(arr) - 1
    while low <= high:
        mid = (low + high) // 2
        if arr[mid] == val:
            return mid
        elif arr[mid] > val:
            high = mid - 1
        else:
            low = mid + 1

assert binary_search([1,2,3,4,5,6], 2) == 1
assert binary_search([1,2,3,4,5,6], 5) == 4
assert binary_search([1,2,3,4,5,6], 7) == -1
<|/ file |>


Наличее тестов явно помогло модели справиться с поставленной задачей лучше.

In [25]:
example = '''\
def <insert>
    """ convert first characters of words to upper case """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def convert_first_char_to_upper(word):
    """ convert first characters of words to upper case """
    word = word.lower()
    return word[0]
<|/ file |>


Упс, что-то пошло не так, попробуем исправить это, для начала не будем использовать тесты, так как уже убедились что они отлично помогают

In [27]:
example = '''\
def first_second_TO_First_Second(text)
    """ convert the first characters of words in a sentence to uppercase """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def first_second_TO_First_Second(text)
    """ convert the first characters of words in a sentence to uppercase """
    words = text.split()
    words[0] = words[0].upper()
    return " ".join(words)
<|/ file |>


Интересный результат, указав пример в названии функции, модель лучше меня поняла, но результат всё равно немного не тот

In [30]:
example = '''\
def <insert>
    """ convert the first characters of words in a sentence to uppercase, the rest of the characters to lowercase """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def convert_first_letters_to_upper(sentence):
    """ convert the first characters of words in a sentence to uppercase, the rest of the characters to lowercase """
    sentence = sentence.split()
    sentence[0] = sentence[0].upper()
    return " ".join(sentence)
<|/ file |>


Более подробное описание задачи приводит к похожему результату, но он всё равно остаётся неверным

In [31]:
example = '''\
def <insert>
    """ convert the first characters of words in a sentence to uppercase, the rest of the characters to lowercase
    
    >>> print(title("aaa bbb ccc"))
    Aaa Bbb Ccc
     """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def title(s):
    """ convert first characters of words in a sentence to uppercase, the rest of the characters to lowercase
    
    >>> print(title("aaa bbb ccc"))
    Aaa Bbb Ccc
     """
    return s[0].upper() + s[1:]

def title2(s):
    """ convert the first characters of words in a sentence to uppercase, the rest of the characters to lowercase
    
    >>> print(title("aaa bbb ccc"))
    Aaa Bbb Ccc
     """
    words = [word.lower() for word in s.split()]
    return " ".join(words)
<|/ file |>


Подсказав ей немного подругому, она выдала несколько вариантов, но опять каждый из них нерабочий, проверим, помогут ли тесты в этот раз

In [32]:
example = '''\
def <insert>
    """ convert the first characters of words in a sentence to uppercase, the rest of the characters to lowercase """
    <insert>


assert f("aaa bbb ccc") == "Aaa Bbb Ccc"
assert f("aaa") == "Aaa"
assert f("aAAbB") == "Aaabb"
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def f(words):
    """ convert the first characters of words in a sentence to uppercase, the rest of the characters to lowercase """
    words = words.split()
    return "".join([word.upper() for word in words])


assert f("aaa bbb ccc") == "Aaa Bbb Ccc"
assert f("aaa") == "Aaa"
assert f("aAAbB") == "Aaabb"
<|/ file |>


Нет, тесты в этот раз не помогли, кажется что модель не знает понятия "слово", потому что в каждом из примеров она пытается работать с входными данными как с "единными" 

Попробуем что-нибудь ещё с использованием сторонних библиотек

In [36]:
example = '''\
def <insert>
    """ geocoding addresses to coordinates """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def geocode(address):
    """ geocoding addresses to coordinates """
    geocoder = pygeocoder.Geocoder()
    location = geocoder.geocode(address)
    return location
<|/ file |>


In [37]:
example = '''\
def <insert>
    """ geocoding coordinates to addresses """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def geocode_to_addresses(lat, lon):
    """ geocoding coordinates to addresses """
    addresses = []
    geocoder = pygeocoder.Geocoder()
    addresses.extend(geocoder.geocode((lat, lon)))
    return addresses
<|/ file |>


Выглядит правдоподобно, но меня смущает выбор библиотеки, есть более популярные варианты

In [45]:
example = '''\
def <insert>
    """ geocoding addresses to coordinates with "GeoPy" library """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def geocode(address):
    """ geocoding addresses to coordinates with "GeoPy" library """
    g = geocoder.google(address)
    return g.latlng
<|/ file |>


Код рабочий, но была выбрана другая библиотека, но не так, которая требовалась, кажется что "GeoPy" не учитывался при обучении модели.

Попробуем задать то, что не решает за несколько строк

In [47]:
example = '''\
def <insert>
    """ parsing the 100 most popular hotels in Rome on TripAdvisor """
    <insert>
<|/ file |>'''
print(docstring_to_code(example))

completed document:
def parse():
    """ parsing the 100 most popular hotels in Rome on TripAdvisor """
    
    hotels = load_hotels()
    
    hotels.sort(key=lambda x: x['popularity'], reverse=True)
    
    return hotels[:100]
<|/ file |>


Выглядит интересно, но это конечно же не то что мы ожидали

# Итоги

На простых запросах InCoder не исптывает проблем. В более сложных задачах результаты выглядят реалистично, но не всегда оказываются рабочими, подсказки в виде наличия тестов или названия функции помогают выдавать результат лучше, но опять же не всегда рабочий.